In [1]:
#From the chapter 14 (Graph.py)
class Graph:
  """Representation of a simple graph using an adjacency map."""

  #------------------------- nested Vertex class -------------------------
  class Vertex:
    """Lightweight vertex structure for a graph."""
    __slots__ = '_element'
  
    def __init__(self, x):
      """Do not call constructor directly. Use Graph's insert_vertex(x)."""
      self._element = x
  
    def element(self):
      """Return element associated with this vertex."""
      return self._element
  
    def __hash__(self):         # will allow vertex to be a map/set key
      return hash(id(self))

    def __str__(self):
      return str(self._element)
    
  #------------------------- nested Edge class -------------------------
  class Edge:
    """Lightweight edge structure for a graph."""
    __slots__ = '_origin', '_destination', '_element'
  
    def __init__(self, u, v, x):
      """Do not call constructor directly. Use Graph's insert_edge(u,v,x)."""
      self._origin = u
      self._destination = v
      self._element = x
  
    def endpoints(self):
      """Return (u,v) tuple for vertices u and v."""
      return (self._origin, self._destination)
  
    def opposite(self, v):
      """Return the vertex that is opposite v on this edge."""
      if not isinstance(v, Graph.Vertex):
        raise TypeError('v must be a Vertex')
      return self._destination if v is self._origin else self._origin
      raise ValueError('v not incident to edge')
  
    def element(self):
      """Return element associated with this edge."""
      return self._element
  
    def __hash__(self):         # will allow edge to be a map/set key
      return hash( (self._origin, self._destination) )

    def __str__(self):
      return '({0},{1},{2})'.format(self._origin,self._destination,self._element)
    
  #------------------------- Graph methods -------------------------
  def __init__(self, directed=False):
    """Create an empty graph (undirected, by default).

    Graph is directed if optional paramter is set to True.
    """
    self._outgoing = {}
    # only create second map for directed graph; use alias for undirected
    self._incoming = {} if directed else self._outgoing

  def _validate_vertex(self, v):
    """Verify that v is a Vertex of this graph."""
    if not isinstance(v, self.Vertex):
      raise TypeError('Vertex expected')
    if v not in self._outgoing:
      raise ValueError('Vertex does not belong to this graph.')
    
  def is_directed(self):
    """Return True if this is a directed graph; False if undirected.

    Property is based on the original declaration of the graph, not its contents.
    """
    return self._incoming is not self._outgoing # directed if maps are distinct

  def vertex_count(self):
    """Return the number of vertices in the graph."""
    return len(self._outgoing)

  def vertices(self):
    """Return an iteration of all vertices of the graph."""
    return self._outgoing.keys()

  def edge_count(self):
    """Return the number of edges in the graph."""
    total = sum(len(self._outgoing[v]) for v in self._outgoing)
    # for undirected graphs, make sure not to double-count edges
    return total if self.is_directed() else total // 2

  def edges(self):
    """Return a set of all edges of the graph."""
    result = set()       # avoid double-reporting edges of undirected graph
    for secondary_map in self._outgoing.values():
      result.update(secondary_map.values())    # add edges to resulting set
    return result

  def get_edge(self, u, v):
    """Return the edge from u to v, or None if not adjacent."""
    self._validate_vertex(u)
    self._validate_vertex(v)
    return self._outgoing[u].get(v)        # returns None if v not adjacent

  def degree(self, v, outgoing=True):   
    """Return number of (outgoing) edges incident to vertex v in the graph.

    If graph is directed, optional parameter used to count incoming edges.
    """
    self._validate_vertex(v)
    adj = self._outgoing if outgoing else self._incoming
    return len(adj[v])

  def incident_edges(self, v, outgoing=True):   
    """Return all (outgoing) edges incident to vertex v in the graph.

    If graph is directed, optional parameter used to request incoming edges.
    """
    self._validate_vertex(v)
    adj = self._outgoing if outgoing else self._incoming
    for edge in adj[v].values():
      yield edge

  def insert_vertex(self, x=None):
    """Insert and return a new Vertex with element x."""
    v = self.Vertex(x)
    self._outgoing[v] = {}
    if self.is_directed():
      self._incoming[v] = {}        # need distinct map for incoming edges
    return v
      
  def insert_edge(self, u, v, x=None):
    """Insert and return a new Edge from u to v with auxiliary element x.

    Raise a ValueError if u and v are not vertices of the graph.
    Raise a ValueError if u and v are already adjacent.
    """
    if self.get_edge(u, v) is not None:      # includes error checking
      raise ValueError('u and v are already adjacent')
    e = self.Edge(u, v, x)
    self._outgoing[u][v] = e
    self._incoming[v][u] = e


In [2]:
#from the chapter 14 (graph_examples.py)

def graph_from_edgelist(E, directed=False):
  """Make a graph instance based on a sequence of edge tuples.

  Edges can be either of from (origin,destination) or
  (origin,destination,element). Vertex set is presume to be those
  incident to at least one edge.

  vertex labels are assumed to be hashable.
  """
  g = Graph(directed)
  V = set()
  for e in E:
    V.add(e[0])
    V.add(e[1])

  verts = {}  # map from vertex label to Vertex instance
  for v in V:
    verts[v] = g.insert_vertex(v)

  for e in E:
    src = e[0] #source
    dest = e[1] #destination
    element = e[2] if len(e) > 2 else None
    g.insert_edge(verts[src],verts[dest],element)

  return g

def simple_example():
    E = (
    ('Seoul', 'Daejon'), ('Daejon','Busan'), ('Busan', 'Daejon'),
    ('Daejon', 'Suwon'), ('Suwon', 'Seoul'), ('Seoul', 'Gwangju'),
    ('Gwangju', 'Suwon'))
    return graph_from_edgelist(E, True)



def figure_14_3():
  """Return the unweighted, directed graph from Figure 14.3 of DSAP."""
  E = (
    ('BOS','SFO'), ('BOS','JFK'), ('BOS','MIA'), ('JFK','BOS'),
    ('JFK','DFW'), ('JFK','MIA'), ('JFK','SFO'), ('ORD','DFW'),
    ('ORD','MIA'), ('LAX','ORD'), ('DFW','SFO'), ('DFW','ORD'),
    ('DFW','LAX'), ('MIA','DFW'), ('MIA','LAX'),
    )
  return graph_from_edgelist(E, True)




In [3]:
graph_korea = simple_example()

In [4]:
#Get the data from incoming (the other is outgoing)
incoming = dict(graph_korea._incoming)

In [24]:
for k,v in incoming.items():
    print (k.element())
    for k1, v1 in v.items():
        print (" ",k1.element())
        print (" ", v1)

LAX
  SFO
  (SFO,LAX,337)
  DFW
  (LAX,DFW,1235)
  MIA
  (LAX,MIA,2342)
DFW
  SFO
  (SFO,DFW,1464)
  LAX
  (LAX,DFW,1235)
  ORD
  (DFW,ORD,802)
  MIA
  (DFW,MIA,1121)
ORD
  SFO
  (SFO,ORD,1846)
  DFW
  (DFW,ORD,802)
  BOS
  (ORD,BOS,867)
  JFK
  (ORD,JFK,740)
JFK
  ORD
  (ORD,JFK,740)
  MIA
  (MIA,JFK,1090)
  BOS
  (JFK,BOS,187)
SFO
  LAX
  (SFO,LAX,337)
  BOS
  (SFO,BOS,2704)
  ORD
  (SFO,ORD,1846)
  DFW
  (SFO,DFW,1464)
BOS
  SFO
  (SFO,BOS,2704)
  ORD
  (ORD,BOS,867)
  MIA
  (MIA,BOS,1258)
  JFK
  (JFK,BOS,187)
MIA
  LAX
  (LAX,MIA,2342)
  DFW
  (DFW,MIA,1121)
  JFK
  (MIA,JFK,1090)
  BOS
  (MIA,BOS,1258)


In [6]:
outgoing = dict(graph_korea._outgoing)

In [7]:
for k,v in outgoing.items():
    print (k.element())
    for k1, v1 in v.items():
        print (" ",k1.element())
        print (" ", v1)

Daejon
  Busan
  (Daejon,Busan,None)
  Suwon
  (Daejon,Suwon,None)
Seoul
  Daejon
  (Seoul,Daejon,None)
  Gwangju
  (Seoul,Gwangju,None)
Gwangju
  Suwon
  (Gwangju,Suwon,None)
Busan
  Daejon
  (Busan,Daejon,None)
Suwon
  Seoul
  (Suwon,Seoul,None)


BOS : Boston
SFO : San Francisco
JFK : John F Kennedy (New York)
MIA : Miami
DFW : Dallas / Fort Worth
LAX : Los Angeles
ORD : O'Hare (Illinois)

In [8]:
graph1 = figure_14_3()

In [9]:
incoming = dict(graph1._incoming)

In [10]:
for k,v in incoming.items():
    print (k.element())
    for k1, v1 in v.items():
        print (" ",k1.element())
        print (" ", v1)

LAX
  DFW
  (DFW,LAX,None)
  MIA
  (MIA,LAX,None)
DFW
  JFK
  (JFK,DFW,None)
  ORD
  (ORD,DFW,None)
  MIA
  (MIA,DFW,None)
ORD
  LAX
  (LAX,ORD,None)
  DFW
  (DFW,ORD,None)
JFK
  BOS
  (BOS,JFK,None)
SFO
  BOS
  (BOS,SFO,None)
  JFK
  (JFK,SFO,None)
  DFW
  (DFW,SFO,None)
BOS
  JFK
  (JFK,BOS,None)
MIA
  BOS
  (BOS,MIA,None)
  JFK
  (JFK,MIA,None)
  ORD
  (ORD,MIA,None)


In [11]:
outgoing = dict(graph1._outgoing)

In [12]:
for k,v in outgoing.items():
    print (k.element())
    for k1, v1 in v.items():
        print (" ",k1.element())
        print (" ", v1)

LAX
  ORD
  (LAX,ORD,None)
DFW
  SFO
  (DFW,SFO,None)
  ORD
  (DFW,ORD,None)
  LAX
  (DFW,LAX,None)
ORD
  DFW
  (ORD,DFW,None)
  MIA
  (ORD,MIA,None)
JFK
  BOS
  (JFK,BOS,None)
  DFW
  (JFK,DFW,None)
  MIA
  (JFK,MIA,None)
  SFO
  (JFK,SFO,None)
SFO
BOS
  SFO
  (BOS,SFO,None)
  JFK
  (BOS,JFK,None)
  MIA
  (BOS,MIA,None)
MIA
  DFW
  (MIA,DFW,None)
  LAX
  (MIA,LAX,None)


In [13]:
def figure_14_14():
  """Return the weighted, undirected graph from Figure 14.14 of DSAP."""
  E = (
    ('SFO', 'LAX', 337), ('SFO', 'BOS', 2704), ('SFO', 'ORD', 1846),
    ('SFO', 'DFW', 1464), ('LAX', 'DFW', 1235), ('LAX', 'MIA', 2342),
    ('DFW', 'ORD', 802), ('DFW', 'MIA', 1121), ('ORD', 'BOS', 867),
    ('ORD', 'JFK', 740), ('MIA', 'JFK', 1090), ('MIA', 'BOS', 1258), 
    ('JFK', 'BOS', 187),
    )
  return graph_from_edgelist(E, False)

In [14]:
graph2 = figure_14_14()

In [15]:
incoming = dict(graph2._incoming)

In [16]:
for k,v in incoming.items():
    print (k.element())
    for k1, v1 in v.items():
        print (" ",k1.element())
        print (" ", v1)

LAX
  SFO
  (SFO,LAX,337)
  DFW
  (LAX,DFW,1235)
  MIA
  (LAX,MIA,2342)
DFW
  SFO
  (SFO,DFW,1464)
  LAX
  (LAX,DFW,1235)
  ORD
  (DFW,ORD,802)
  MIA
  (DFW,MIA,1121)
ORD
  SFO
  (SFO,ORD,1846)
  DFW
  (DFW,ORD,802)
  BOS
  (ORD,BOS,867)
  JFK
  (ORD,JFK,740)
JFK
  ORD
  (ORD,JFK,740)
  MIA
  (MIA,JFK,1090)
  BOS
  (JFK,BOS,187)
SFO
  LAX
  (SFO,LAX,337)
  BOS
  (SFO,BOS,2704)
  ORD
  (SFO,ORD,1846)
  DFW
  (SFO,DFW,1464)
BOS
  SFO
  (SFO,BOS,2704)
  ORD
  (ORD,BOS,867)
  MIA
  (MIA,BOS,1258)
  JFK
  (JFK,BOS,187)
MIA
  LAX
  (LAX,MIA,2342)
  DFW
  (DFW,MIA,1121)
  JFK
  (MIA,JFK,1090)
  BOS
  (MIA,BOS,1258)


# Depth-First Search

In [17]:
#From Chapter 14 (dfs.py)
def DFS(g, u, discovered):
  """Perform DFS of the undiscovered portion of Graph g starting at Vertex u.

  discovered is a dictionary mapping each vertex to the edge that was used to
  discover it during the DFS. (u should be "discovered" prior to the call.)
  Newly discovered vertices will be added to the dictionary as a result.
  """
  for e in g.incident_edges(u):    # for every outgoing edge from u
    v = e.opposite(u)
    if v not in discovered:        # v is an unvisited vertex
      discovered[v] = e            # e is the tree edge that discovered v
      DFS(g, v, discovered)        # recursively explore from v

def construct_path(u, v, discovered):
  """
  Return a list of vertices comprising the directed path from u to v,
  or an empty list if v is not reachable from u.

  discovered is a dictionary resulting from a previous call to DFS started at u.
  """
  path = []                        # empty path by default
  if v in discovered:
    # we build list from v to u and then reverse it at the end
    path.append(v)
    walk = v
    while walk is not u:
      e = discovered[walk]         # find edge leading to walk
      parent = e.opposite(walk)
      path.append(parent)
      walk = parent
    path.reverse()                 # reorient path from u to v
  return path

def DFS_complete(g):
  """Perform DFS for entire graph and return forest as a dictionary.

  Result maps each vertex v to the edge that was used to discover it.
  (Vertices that are roots of a DFS tree are mapped to None.)
  """
  forest = {}
  for u in g.vertices():
    if u not in forest:
      forest[u] = None             # u will be the root of a tree
      DFS(g, u, forest)
  return forest


In [18]:
dfs_incoming_simple = DFS_complete(graph_korea)

In [19]:
for k, v in dfs_incoming_simple.items():
    print (k.element())
    print (" ",v)

Daejon
  None
Busan
  (Daejon,Busan,None)
Suwon
  (Daejon,Suwon,None)
Seoul
  (Suwon,Seoul,None)
Gwangju
  (Seoul,Gwangju,None)


In [20]:
dfs_incoming = DFS_complete(graph1) #, list_incoming[0])

In [21]:
for k, v in dfs_incoming.items():
    print (k.element())
    print (" ",v)

LAX
  None
ORD
  (LAX,ORD,None)
DFW
  (ORD,DFW,None)
SFO
  (DFW,SFO,None)
MIA
  (ORD,MIA,None)
JFK
  None
BOS
  (JFK,BOS,None)


In [22]:
dfs_incoming2 = DFS_complete(graph2)

In [23]:
for k, v in dfs_incoming2.items():
    print (k.element())
    print (" ",v)

LAX
  None
SFO
  (SFO,LAX,337)
BOS
  (SFO,BOS,2704)
ORD
  (ORD,BOS,867)
DFW
  (DFW,ORD,802)
MIA
  (DFW,MIA,1121)
JFK
  (MIA,JFK,1090)
